## Introduction to our first task: 'Dogs vs Cats'

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
PATH = "data/dogscats/"
sz=224
batch_size=64

In [32]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input
from keras.models import model_from_json
import cv2
import os

In [36]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'
test_dir=f'{PATH}test/test1/'

In [18]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode="categorical")

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

In [ ]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])    

In [ ]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=1, workers=3,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [6]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
import glob
from PIL import Image
import pickle
import cv2
import numpy as np
filelist = glob.glob('data/dogscats/test/test1/*.jpg')

In [ ]:
x1 = np.array([np.array(Image.open(f'{PATH}test/test1/1.jpg'))])

In [8]:
test_generator = test_datagen.flow_from_directory(test_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=64, class_mode="input")
images,labels = next(iter(test_generator))

Found 12500 images belonging to 1 classes.


In [10]:
test_generator.reset()
predictions = loaded_model.predict_generator(
test_generator,
steps=int(test_generator.samples/float(batch_size)), # all samples once
verbose = 1,
workers = 2,
max_q_size=10,
pickle_safe=True
)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

/home/vaibhavnandwani/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `predict_generator` call to the Keras 2 API: `predict_generator(<keras.pre..., steps=195, verbose=1, workers=2, use_multiprocessing=True, max_queue_size=10)`
  


195/195 [==============================] - 112s 576ms/step


In [12]:
predictions[:10]

array([[9.9999309e-01],
       [6.0792960e-04],
       [2.9808742e-05],
       [9.9999952e-01],
       [1.0000000e+00],
       [1.3253603e-05],
       [1.9885068e-05],
       [9.9999535e-01],
       [9.9936587e-01],
       [3.9034917e-06]], dtype=float32)

In [25]:
print (test_generator.filenames[0:10])
test_generator.

['test1/1.jpg', 'test1/10.jpg', 'test1/100.jpg', 'test1/1000.jpg', 'test1/10000.jpg', 'test1/10001.jpg', 'test1/10002.jpg', 'test1/10003.jpg', 'test1/10004.jpg', 'test1/10005.jpg']


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [ ]:
loaded_model.predict(images)

In [ ]:
pred=[]
for file in filelist[:5]:
    images = np.array([np.array(Image.open(file))])
    pred.append(loaded_model.predict(images))

In [ ]:
pred

In [ ]:
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')


In [ ]:
from matplotlib import pyplot as plt

plots(images)

In [ ]:
from IPython.display import Image
Image("data/dogscats/test/test1/10000.jpg")

In [44]:
def prepare_data(list_of_images):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels
    
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (150,150), interpolation=cv2.INTER_CUBIC))
    
    for i in list_of_images:
        if 'dog' in i:
            y.append(1)
        elif 'cat' in i:
            y.append(0)

In [47]:
X_test, Y_test = prepare_data(test_images_dogs_cats)

TypeError: 'NoneType' object is not iterable

In [46]:
test_images_dogs_cats = [test_dir+i for i in os.listdir(test_dir)]